<img style="float: right; padding: 10px;" width="200" src="images/logo-onem2m-the-iot-standard.jpg">


# oneM2M - Access Control

This notebook demonstrates how access control to resources can be done in oneM2M.

- Create an &lt;ACP> resource with different credentials for a new originator
- Create a second &lt;AE> resource with the new access controls policy
- Succeed to add a &lt;container> to the second &lg;AE> resource
- Fail to update the second &lt;AE> resource
 
 
<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    In the previous notebooks we always used the originator <i>Cmyself</i> to create resources under the &lt;AE> resource for that originator. This succeedded because, if no other restrictions apply, the originator can always access the resources under its own &lt;AE> resource. 
<br><br>
This notebook gives a short overview about how access to resources can be restricted and controlled.
    </div>
</div>
</p>
 


## Intitialization
The section does import necessary modules and configurations, and prepares the CSE for this notebook.

In [ ]:
%run src/init.py acp


## Create an &lt;ACP> Resource

<img align="right" src="images/05/create_acp.png">


Access Control Policies are used to associate access control with credentials. They define the rules to for access control to resources. Each &lt;ACP> resource has two sections:

- **pv (Privileges)** : The actual privileges defined by this policy.
- **pvs (Self-Privileges)** : This defines the privileges necessary to access and control the &lt;ACP> resource itself.

Each section define, at least, the request originators and the allowed operations. Other settings here could be date, time, geo-location, network addresses, and more.


The following request creates a new &lt;ACP> that allows the originator "Clegal" only to send CREATE, RETRIEVE, NOTIFY, DELETE, and DISCOVERY (**but not UPDATE!**) requests to resources that have this &lt;ACP> resource assigned.

In [ ]:

CREATE (                                       # CREATE request
    
    # Create the ACP resource under the CSEBase
    to                      = cseBaseName,  
    
    # Request Parameters
    originator              = 'Cmyself',                            # Assign an originator ID, must start with 'C'
    requestIdentifier       = '123',                                # Unique request identifier
    releaseVersionIndicator = '3',                                  # Release version indicator
    resourceType            = Type.ACP,                             # Type of the resource: Access Control Policy
    
    # Request Body
    primitiveContent = 
        {
            'm2m:AccessControlPolicy': {
                'resourceName' : 'Notebook-ACP',                    # Name of the resource 
                'privileges' : {                                    # Privileges used in access to other resources
                    'accessControlRule': [ {                        # List of access control rules
                        'accessControlOriginators': [ 'Clegal' ],   # Access control originator
                        'accessControlOperations': 59               # Access control operations (bitfield): 
                                                                    # CREATE, RETRIEVE, DELETE, NOTIFY, DISCOVERY, but not UPDATE
                    }]
                },
                'selfPrivileges': {                                 # Privileges used in access to this <ACP> resource
                    'accessControlRule' : [{                        # List of access control rules
                        'accessControlOriginators' : [ 'Cmyself' ], # Access control originator
                        'accessControlOperations' : 63              # Access control operations (bit-field): ALL
                    }]
                }
            }
        }

)


### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute                | Short Name | Description                                                                                                                               |
|:-------------------------|:-----------|:------------------------------------------------------------------------------------------------------------------------------------------|
| privileges               | pv         | The privileges for other originators defined by this access control policy.                                                               |
| selfPrivileges           | pvs        | The privileges necessary to access and control this &lt;ACP> resource itself.                                                             |
| accessControlRule        | acr        | A set of control rules for the privilegs                                                                                                  |
| accessControlOriginators | acor       | This list includes the *originator* information. The parameter comprises a list of domain, CSE-IDs, AE-IDs, and other oneM2M identifiers. |
| accessControlOperations  | acop       | This number represents a bit-field of granted privileges. The following table shows the mapping.                                          |


#### Access Control Operations
| Value | Interpretation |
|:------|:---------------|
| 1     | CREATE         |
| 2     | RETRIEVE       |
| 4     | UPDATE         |
| 8     | DELETE         |
| 16    | NOTIFY         |
| 32    | DISCOVERY      |



<img align="right" src="images/05/create_ae2.png">


## Create a second &lt;AE> Resource with the new &lt;ACP> assigned

We now create a new &lt;AE> resource that refers to the just created &lt;ACP>.  This will allow an originator *Clegal* to send the selected request operations to that AE resource.

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
We let the CSE assign a new Originator to the AE. This is done by just assigning "C" to the <i>originator</i> in the request. The CSE then generates and assigns a new random originator to the AE. This is returned in the request's response in the <i>aei</i> (APP-ID) attribute.
   
For this example we don't care about the value of this originator, because we want to demonstrate that one will get access to the resource via the Access Control Policies.
        
Note the *accessControlPolicyID* attribute. This attribute contains the resource ID(s) to &lt;AccessControlPolicy> resources that will from now on control the access to this resource.
    </div>
</div>
</p>


**<font color="green">This request should succeed.</font>**

In [ ]:
CREATE (                                              # CREATE request
    
    # Create the AE resource under the CSEBase
    to                      = cseBaseName,  
    
    # Request Parameters
    originator              = 'C',                    # Ask the CSE to assign a random originator
    requestIdentifier       = '123',                  # Unique request identifier
    releaseVersionIndicator = '3',                    # Release version indicator
    resourceType            = Type.AE,                # Type of the resource: AE
    
    # Request Body
    primitiveContent = 
        {   
            'm2m:ApplicationEntity': {
                'resourceName':  'Notebook-AE_2',     # Name of the resource 
                'App-ID': 'NnotebookAE2',             # Application ID, must start with 'N'
                'requestReachability':  True,         # AE can receive requests
                'supportedReleaseVersions': [ '3' ],  # Supports oneM2M release 3
                'accessControlPolicyIDs' : [ 
                    cseBaseName + '/Notebook-ACP'     # Assign the <ACP> resource to this resource
                ]
            }
        }
)


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    Note, that the &lt;ACP>'s resource identifier in the <i>acpi</i> attribute has been changed to unstructured format.
    </div>
</div>
</p>


### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute               | Short Name | Description                                                           |
|:------------------------|:-----------|:----------------------------------------------------------------------|
| accessControlPolicyIDs  | acpi       | A list of resource IDs that will control the access to the resource.  |


<img align="right" src="images/05/create_container.png">

## Try to Create a &lt;container> under the second &lt;AE> Resource

We will create  a &lt;xontainer> resource under the second &lt;AE> "Notebook-AE_2" resource with the originator of "Clegal". 

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
    The request originator <i>Clegal</i> is <u>allowed</u> by the &lt;ACP> resource to send CREATE requests to the second AE.
    <br>
    Also note, that this resource will have no &lt;ACP> resource assigned (in the <i>accessControlPolicyIDs<i> attribute).
    </div>
</div>
</p>


**<font color="green">This will also work</font>** 


In [ ]:
CREATE (                                        # CREATE request
    
    # Create the container resource under the second AE
    to                      = cseBaseName + '/Notebook-AE_2',  
    
    # Request Parameters
    originator              = 'Clegal',         # Set the originator
    requestIdentifier       = '123',            # Unique request identifier
    releaseVersionIndicator = '3',              # Release version indicator
    resourceType            = Type.Container,   # Type of the resource: Container
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Container' : {
                'resourceName'  : 'Container'
            }
        }
)

<img align="right" src="images/05/update_container.png">


## Try to update the new &lt;container> Resource

Next, we will try to update the just created &lt;container> resource (add a *lbl* attribute) with the same originator "Clegal". 

**<font color="green">This will succeed as well</font>**

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
With most resource types that <i>could</i> have an <i>acpi</i> attribute but have none set, the original creator of that resource has all access to that resource by default. 
    
In our case this means that the AE under which a resource has been created <u>does not</u> have access to that resource, because it is not the creator. Access must be set explicitly in this case to restrict further access.
    </div>
</div>
</p>


In [ ]:
UPDATE (                                        # UPDATE request
    
    # Create the container resource under the second AE
    to                      = cseBaseName + '/Notebook-AE_2/Container',  
    
    # Request Parameters
    originator              = 'Clegal',         # Set the originator
    requestIdentifier       = '123',            # Unique request identifier
    releaseVersionIndicator = '3',              # Release version indicator
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Container' : {
                'labels': [ 'test:test' ]
            }
        }
)

<img align="right" src="images/05/update_ae2.png">

## Try to update the second &lt;AE> Resource

Now we will try to update the second &lt;AE> resource (here we add a *lbl* attribute) with the same originator "Clegal". 

**This should fail!**
 
<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 350px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
        Remember, the associated &lt;ACP> resource <u>doesn't</u> allow UPDATE requests to this resource for <i>Clegal</i>.
    </div>
</div>
</p>

In [ ]:
UPDATE (                                        # UPDATE request
    
    # Create the container resource under the second AE
    to                      = cseBaseName + '/Notebook-AE_2',  
    
    # Request Parameters
    originator              = 'Clegal',         # Set the originator
    requestIdentifier       = '123',            # Unique request identifier
    releaseVersionIndicator = '3',              # Release version indicator
    
    # Request Body
    primitiveContent = 
        {
            'm2m:ApplicationEntity' : {
                'labels': [ 'test:test' ]
            }
        }
)


### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute  | Short Name | Description                                                                   |
|:-----------|:-----------|:------------------------------------------------------------------------------|
| debugInfo  | dbg        | An optional error message than contains further information about the error.  |
